In [ ]:
# !pip install corus
# !pip install razdel
# !wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

In [9]:
import corus

from razdel import tokenize

In [10]:
path = 'Persons-1000.zip'
records = corus.persons.load_persons(path)
rec = next(records)
rec

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [11]:
word_tag = []

for rec in records:
  for r in rec.spans:
    for t in tokenize(rec.text):
      if (t.start >= r.start) and (t.stop <= r.stop):
        word_tag.append((rec.text[t.start:t.stop], 'PERSON'))
      else:
        word_tag.append((rec.text[t.start:t.stop], 'NO_PERSON'))

print(len(word_tag)) 
print(word_tag[:5])

3715842
[('Комиссар', 'NO_PERSON'), ('СЕ', 'NO_PERSON'), ('критикует', 'NO_PERSON'), ('ограничительную', 'NO_PERSON'), ('политику', 'NO_PERSON')]


In [12]:
for i in word_tag[:1000]:
  if i[1] == 'PERSON':
    print(i)

('Томас', 'PERSON')
('Хаммарберг', 'PERSON')
('(', 'PERSON')
('Thomas', 'PERSON')
('Hammarberg', 'PERSON')
(')', 'PERSON')
('Хаммарберг', 'PERSON')
('Михаила', 'PERSON')
('Саакашвили', 'PERSON')
('Егора', 'PERSON')
('Борисова', 'PERSON')
('Егора', 'PERSON')
('Борисова', 'PERSON')


# Проверить насколько хорошо работает NER

## 1. взять нер из nltk

In [5]:
import nltk
nltk.download('words')
nltk.download('maxent_ne_chunker')
nltk.download('names')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [32]:
{(' '.join(c[0] for c in chunk), chunk.label()) for chunk in nltk.ne_chunk(word_tag[:200_000]) if hasattr(chunk, 'label')}

{('AP', 'ORGANIZATION'),
 ('Clinton', 'PERSON'),
 ('HSBC', 'ORGANIZATION'),
 ('John', 'PERSON'),
 ('John Huntsman Jr', 'PERSON'),
 ('John Roos', 'PERSON'),
 ('New York', 'GPE'),
 ('Washington', 'GPE'),
 ('Washington Post', 'ORGANIZATION'),
 ('САО', 'ORGANIZATION')}

#### Вывод:
 - ne_chunk не распознает имена на кирилице
 - однако он распознает организации на кирилице

## 2. проверить deeppavlov

In [ ]:
# !python -m venv env
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert
# !python -m deeppavlov install ner_ontonotes

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)

2021-04-10 16:19:31.411 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/kbqa/datasets/entity_and_type_detection_rus.pickle to /root/.deeppavlov/models/entity_and_type_detection_rus.pickle
100%|██████████| 2.07M/2.07M [00:00<00:00, 5.60MB/s]
2021-04-10 16:19:32.412 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz to /root/.deeppavlov/models/ner_cq_rus.tar.gz
100%|██████████| 1.32G/1.32G [02:05<00:00, 10.5MB/s]
2021-04-10 16:21:38.381 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/models/ner_cq_rus.tar.gz archive into /root/.deeppavlov/models/ner_ent_and_type_rus
2021-04-10 16:21:56.369 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|███████

2021-04-10 16:23:03.936 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ent_and_type_rus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-04-10 16:23:35.594 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ent_and_type_rus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ent_and_type_rus/model


In [ ]:
deeppavlov_ner(word_tag[:10000])

#### Вывод:
- на деппавлове падает колаб

## 3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них

In [75]:
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [66]:
X = []
y = []

for tok, lbl in word_tag:
  X.append(tok)
  y.append(lbl)

X[:5], y[:5]

(['Комиссар', 'СЕ', 'критикует', 'ограничительную', 'политику'],
 ['NO_PERSON', 'NO_PERSON', 'NO_PERSON', 'NO_PERSON', 'NO_PERSON'])

In [67]:
le = LabelEncoder()
y_train_test = le.fit_transform(y)
print(le.classes_)

['NO_PERSON' 'PERSON']


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y_train_test, test_size=0.3)

In [69]:
hvec_ch = HashingVectorizer(ngram_range=(1,5), analyzer='char_wb', n_features=100)

In [70]:
X_train_vec = hvec_ch.fit_transform(X_train)
X_test_vec = hvec_ch.transform(X_test)

In [74]:
lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [77]:
roc_auc_score(y_test, y_pred)

0.5

#### Выводы:
- roc_auc_score показал, что шанс верного угадывания персоны случаен

#### 4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

- NER из nltk хорошо угадывает персон, чьи имена написаны латиницей
- Кастомный NER показал плохой результат